In [1]:
import pandas as pd
import numpy as np

In [2]:
debug=0

⚠️ Attention à fournir un fichier XLSX

In [6]:
# notesNom=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/19-LNS1U4-LNS1T2-eCampus-Majeure.xlsx"
# notesNom=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/19-Session2-LNS1U4-LNS1T2-eCampus-Mineures-Final.xlsx"
# notesNom=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/19-LNS1U4-LNS1T2-eCampus-Mineures.xlsx"
# notesNom=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/19-Session2-LNS1U4-LNS1T2-eCampus-Majeure.xlsx"
# notesNom=u"/Users/gilles/Downloads/20-LNS1T2-Mineures.xlsx"
# notesNom=u"/Users/gilles/Downloads/20-LNS1T2-FAD.xlsx"
# notesNom=u"/Users/gilles/Downloads/20-LNS1U4-Majeure.xlsx"
# notesNom=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/NotesCours/21-Session1/21-LNS1U4-LNS1T2-eCampus-Mineures.xlsx"
# notesNom=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/NotesCours/21-Session1/21-LNS1U4-LNS1T2-eCampus-FAD.xlsx"
# notesNom=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/NotesCours/21-Session1/21-LNS1U4-LNS1T2-eCampus-Majeure.xlsx"
notesNom=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/NotesCours/23-Session1/23-1LNSU4.xlsx"

In [35]:
xls = pd.ExcelFile(notesNom)
xls.sheet_names

[u'Notes']

In [17]:
# dfNotes=pd.read_excel(notesNom,encoding="utf8")
dfNotes = pd.read_excel(xls, sheet_name=u'Notes',encoding="utf8")
dfNotes.set_index([u"Numéro d'identification"],inplace=True)
len(dfNotes.columns)

23

In [23]:
dfNotes.columns=[u'Nom', u'Prénom', u'Adresse de courriel',u'Prasi', u'Yomig V', u'Yomig T', u'Kartin V', u'Kartin T', u'Nemas V', u'Nemas T', u'CoupeMot', u'CoupeMorphe']+sorted([u"Q%02d"%(n+1) for n in range(1,11)])+["date"]

# dfNotes["Q02"]

#### sélection des colonnes
- notesCC pour le contrôle continu
- notesQuizz pour les quizz en CM

In [25]:
notesCC=[u'Prasi', u'Yomig V', u'Yomig T', u'Kartin V', u'Kartin T', u'Nemas V', u'Nemas T',
         u'CoupeMot', u'CoupeMorphe'
        ]
# notesCC=u"Morpho1 Morpho2 Bizar1 Bizar2 Phono Sem".split(" ")
# notesQuizz=sorted([u"Quizz%d"%(n+1) for n in range(2,8)]+[u"Quizz7b"])
notesQuizz=sorted([u"Q%02d"%(n+1) for n in range(1,11)])
# dfNotes=dfNotesXLS[notesCC+notesQuizz]
dfNotes[notesQuizz].head(10)

,Q02,Q03,Q04,Q05,Q06,Q07,Q08,Q09,Q10,Q11
Numéro d'identification,,,,,,,,,,
436893,19,14,13,13,7,7,20,17,15,12
440340,17,13,14,14,11,14,17,18,12,17
435916,14,14,16,17,14,16,15,18,13,13
442931,13,13,14,13,10,11,-,-,-,-
438199,19,13,0,13,20,14,13,13,13,13
386382,19,19,-,-,-,-,-,-,-,-
440881,19,19,16,17,20,16,15,17,17,15
441950,-,16,-,-,-,-,-,-,-,-
437791,16,13,14,17,20,0,15,13,17,15


### Calcul des notes avec bonification
Toutes les notes du CC (*nbCC*) comptent et on inclue aussi les *nbCC* meilleures notes des Quizz en partant de la meilleure une par une
- *notes* contient toutes les moyennes calculées
- on retourne le max de *notes*

In [26]:
def moyenneCC(x):
    if debug: print x.name
    xNotesCC=sorted([n for n in x[notesCC].tolist() if not (n=="-" or np.isnan(n))],reverse=True)
    sommeNotes=float(sum(xNotesCC))
    nbNotes=len(notesCC)
    moyenneInitiale=round(sommeNotes/nbNotes+.25)
    return moyenneInitiale
        

In [27]:
def bonus(x):
    if debug: print x.name
    xNotesCC=sorted([n for n in x[notesCC].tolist() if not (n=="-" or np.isnan(n))],reverse=True)
    sommeNotes=float(sum(xNotesCC))
    nbNotes=len(notesCC)
    moyenneInitiale=round(sommeNotes/nbNotes+.25)
    xNotesBonus=sorted([n for n in x[notesQuizz].tolist() if not (n=="-" or np.isnan(n))],reverse=True)
    if debug: print "Quizz",xNotesBonus
    if debug: print "CC",xNotesCC
    if xNotesCC:
        notes=[]
        nbCC=len(xNotesCC)
        bonusNotes=xNotesBonus
        if len(bonusNotes)>nbCC:
            bonusNotes=bonusNotes[:nbCC]
        notes.append(moyenneInitiale)
        if debug: print "somme", sommeNotes,"moyenne brut",sommeNotes/nbNotes
        if debug: print "moyenne initiale",moyenneInitiale,"bonus",bonusNotes
        for b in bonusNotes:
            sommeNotes+=b
            nbNotes+=1
            nouvelleMoyenne=round(float(sommeNotes)/nbNotes+.25)
            notes.append(nouvelleMoyenne)
            if debug: print b,sommeNotes,nbNotes, nouvelleMoyenne
        if debug: print notes
        return max(notes)
    else:
        return np.nan
        

### Ajout de la colonne **Notes** à la structure et exportation
- apply(lambda x: bonus(x), axis=1) permet d'appliquer la fonction bonus à chaque ligne
 - la fonction bonus reçoit une ligne de données complète
 - les valeurs sont accessibles par colonne
- on ajoute 3 colonnes
 - Note pour la note de contrôle continu avec les bonus de quizz
 - NoteCC pour la note des exercices en ligne brute
 - Gain pour la différence entre les deux notes

In [28]:
dfNotes.loc[:,"Note"]=dfNotes.loc[:,notesCC+notesQuizz].apply(lambda x: bonus(x),axis=1)
dfNotes.loc[:,"NoteCC"]=dfNotes.loc[:,notesCC].apply(lambda x: moyenneCC(x),axis=1)

In [29]:
dfNotes["Gain"]=dfNotes["Note"]-dfNotes["NoteCC"]

In [30]:
dfNotes["Gain"].mean()

1.9836065573770492

In [33]:
dfNotes[dfNotes["Note"].notnull()][u"Prénom Nom Note NoteCC Gain".split(" ")]

,Prénom,Nom,Note,NoteCC,Gain
Numéro d'identification,,,,,
436893,GABRIEL,ABELA--FRANZETTI,12.0,10.0,2.0
440340,FLORELLE,ABERARD,14.0,12.0,2.0
435916,EULALIE,ADAM,14.0,13.0,1.0
438199,NESLIHAN,AKPINAR,12.0,10.0,2.0
440881,RACHEL,AMOUZOU,14.0,10.0,4.0
437791,MAURINE,APPRIOU,17.0,17.0,0.0
440486,ELYA,AURICES,11.0,8.0,3.0
436384,CAMILLE,BALDY- - BOUILLEUX,17.0,16.0,1.0
442845,SOUHA,BEN HAMOUDA,4.0,2.0,2.0


In [40]:
with pd.ExcelWriter(notesNom,mode='a',engine='openpyxl') as writer: 
    dfNotes[u"Prénom Nom Note NoteCC Gain".split(" ")].to_excel(writer, sheet_name=u'eCampus',encoding="utf8")